**Expert Knowledge Worker**

A question answering agent that is an expert knowledge worker
To be used by employeed of insurellm, an Insurance Tech company
The agent needs to be acurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

The First implimentation will use a simple , brute-force type of RAG.

In [73]:
import os
import glob
from dotenv import load_dotenv, find_dotenv
import gradio as gr
from pathlib import Path
from langchain_openai import ChatOpenAI

In [74]:
MODEL = "gpt-4o-mini"

In [ ]:
load_dotenv(find_dotenv(), override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

MODEL = "gpt-4o-mini"
chatOpenAI = ChatOpenAI(model=MODEL, temperature=0.7, streaming=True)

In [76]:
knowledge = {}
employees = glob.glob("./knowledge-base/employees/*")

for employee in employees:
    name = employee.split(" ")[-1][:-3]
    doc = ""
    with open(employee, "r", encoding="utf-8") as f:
        knowledge[name] = f.read()



In [77]:
knowledge.keys()

dict_keys(['Chen', 'Harper', 'Thomson', 'Foster', 'Lancaster', 'Walker', 'Rodriguez', 'Park', 'Kim', 'Carter', 'Tran', 'Wilson', 'Adams', 'Liu', 'Blake', 'Bishop', 'Zhang', 'Anderson', 'Johnson', 'Thompson', "O'Brien", 'Rivera', 'Patel', 'Spencer', 'Sharma', 'Martinez', 'Greene', 'Trenton', 'Williams', 'Brooks'])

In [78]:
products = glob.glob('knowledge-base/products/*')
for product in products:
    name = Path(product).stem
    doc = ""
    with open(product, "r", encoding="utf-8") as f:
        knowledge[name] = f.read()
        

In [79]:
knowledge['Bizllm']

"# Product Summary\n\n# Bizllm\n\n## Summary\n\nBizllm is Insurellm's enterprise-grade commercial insurance platform designed to revolutionize how insurers serve business customers. From small businesses to large corporations, Bizllm provides comprehensive tools for underwriting, policy administration, and risk management across multiple commercial lines including general liability, professional liability, property, workers' compensation, and cyber insurance. By leveraging AI and industry-specific data analytics, Bizllm enables commercial insurers to assess complex risks accurately, price policies competitively, and deliver exceptional service to business clients.\n\n## Features\n\n### 1. Multi-Line Underwriting Engine\nBizllm's sophisticated underwriting platform handles diverse commercial insurance products within a single unified system. AI-powered risk assessment evaluates business operations, industry-specific exposures, and financial stability to generate accurate quotes across m

In [80]:
system_mesage = """
You represent Insurellm, the Insurance Tech company.
You are an expert in answering questions about Insurellm; its employees and its products.
You are provided with additional context that might be relevant to the user's question.
Give brief, accurate answers. If you don't know the answer, say so.
Relevant context:
"""

In [81]:
def get_relevant_context(message):
    relevant_knowledge = []
    for knowledge_title, knowledge_details in knowledge.items():
        # message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        if knowledge_title in message:
            relevant_knowledge.append(knowledge_details)
    return relevant_knowledge


In [82]:
get_relevant_context("Who is Avery Lancaster and what is carllm?")

["# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985\n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)\n- **Location**: San Francisco, California\n- **Current Salary**: $225,000  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market 

In [59]:
def add_context(message):
    relevant_knowledge = get_relevant_context(message)
    if relevant_knowledge:
        message += "\n\nThe following additional context might be relevant in anwering the question:\n\n"
        for relevant in relevant_knowledge:
            message += relevant + "\n\n"
    return message

In [83]:
print(add_context("Who is Avery Lancaster?"))

Who is Avery Lancaster?

The following additional context might be relevant in anwering the question:

# Avery Lancaster

## Summary
- **Date of Birth**: March 15, 1985
- **Job Title**: Co-Founder & Chief Executive Officer (CEO)
- **Location**: San Francisco, California
- **Current Salary**: $225,000  

## Insurellm Career Progression
- **2015 - Present**: Co-Founder & CEO  
  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  

- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  
  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  

- **2010 - 2013**: Business Analyst at Edge Analytics  

In [ ]:
def chat(message, history):
    '''
    Gradio passes: 
    message: latest user message (str)
    history: list[dict] of {"role": ..., "content": ...}
    '''
    # Start with the system message

    messages = [{"role": "system", "content": system_mesage}]

    # Append the prior conversation

    for h in history:
        messages.append({"role": h["role"], "content": h["content"]})
    
    user_input = add_context(message)
    # Append the latest user input
    messages.append({"role": "user", "content": user_input})

    # Stream Response
    response = ""
    updated_history = history + [{"role": "user", "content": message}]  # show plain user input

    for chunk in chatOpenAI.stream(messages):
        if chunk.content:
            response +=chunk.content
            yield updated_history + [{"role": "assistant", "content": response}]    




In [89]:
with gr.Blocks() as demo:
    gr.Markdown("### 🤖 InsureLLM AI Assistant")
    chatbot = gr.Chatbot(type="messages", label="AI Assistant", height=500)
    msg = gr.Textbox(placeholder="Type your message...", label="Your Message")
    msg.submit(chat, [msg, chatbot], [chatbot])
    msg.submit(chat, [msg, chatbot], [chatbot]).then(
    lambda: "", None, [msg]
    )#clear text box after submit

demo.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
